<a href="https://colab.research.google.com/github/tinkvu/TravelRecommenderUsingLLM/blob/main/Notebooks/Dublin_Tourist_%2B_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/tinkvu/TravelRecommenderUsingLLM/
!ls

Cloning into 'TravelRecommenderUsingLLM'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 95 (delta 30), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (95/95), 1.81 MiB | 3.39 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Data  sample_data  TravelRecommenderUsingLLM


#Data Cleaning

In [13]:
#import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import string

from wordcloud import WordCloud
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [9]:
import pandas as pd

# Read the CSV file and drop the first column
df = pd.read_csv('/content/TravelRecommenderUsingLLM/Raw Data/Processed/processed_datas.csv').iloc[:, 1:]

# Print the DataFrame
df.head()

,place_name,rating,review
0,14 Henrietta Street,4.875,The museum is very good. I highly recommend it...
1,1916 Rebellion Walking Tour,4.975,"Lorcan is a professional guide, passionate abo..."
2,3fe Grand Canal Street,4.700,Delicious coffee and breakfast in Dublin. Lots...
3,Arthur Guinness Dwelling House,5.000,Who would think Guinness Beer at that time?
4,As One,4.450,Came here on a Saturday morning without a book...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   place_name  217 non-null    object 
 1   rating      217 non-null    float64
 2   review      216 non-null    object 
dtypes: float64(1), object(2)
memory usage: 5.2+ KB


In [15]:
# Downloading the NLTK stopwords dataset.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize NLTK's English stopwords and WordNet lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to clean and normalize text
def clean_text(text):
    if isinstance(text, str):  # Check if the input is a string
        text = text.lower()  # Lowercase
        text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        tokens = word_tokenize(text)  # Tokenize text into words
        tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]  # Remove stopwords and lemmatize
        cleaned_text = ' '.join(tokens)  # Join tokens back into a single string
        return cleaned_text
    else:
        return ''  # Return empty string for non-string inputs (like NaN)

# Apply text cleaning
df['review1'] = df['review'].apply(clean_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
df.tail()

,place_name,rating,review,review1
212,Wellington Monument,4.625,Very impressive monument and great place to re...,impressive monument great place relax grass su...
213,White Rabbit Cafe,4.200,Theres this really cool little coffee shop on ...,there really cool little coffee shop busy stre...
214,Wolfe Tone Park,3.925,A popular little square with seating and grass...,popular little square seating grassed area the...
215,World Poverty Stone,4.450,Impressive lean face people figures made of st...,impressive lean face people figure made stone ...
216,"laine, my love",4.350,Nice use of a small space. Really affordable a...,nice use small space really affordable good po...


In [22]:
df=df.drop(['review','rating'],axis=1)

In [27]:
df['review']=df['review1']
df=df.drop(['review1'],axis=1)

In [28]:
df.head()

,place_name,review
0,14 Henrietta Street,museum good highly recommend know georgian hou...
1,1916 Rebellion Walking Tour,lorcan professional guide passionate irish his...
2,3fe Grand Canal Street,delicious coffee breakfast dublin lot indoor s...
3,Arthur Guinness Dwelling House,would think guinness beer time
4,As One,came saturday morning without booking got tabl...


In [29]:
df.to_csv('cleaned_data.csv',index=False)

#Try1

In [11]:
!python -m pip install --upgrade pip
!pip3 install torch==2.2.1 torchvision torchaudio   xformers --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft \
    accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps  trl peft accelerate bitsandbytes
!pip install datasets
!pip install hyperopt
!pip install optuna
pass

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-e9jleyr1/unsloth_47fc969531f644eaaa7ae38b6668b5c2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-e9jleyr1/unsloth_47fc969531f644eaaa7ae38b6668b5c2
  Resolved https://github.com/unslothai/unsloth.git to commit 499635a19013319367358dcf2a0211d30073ebd0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!python -m xformers.info
!python -m bitsandbytes
!nvidia-smi

Unable to find python bindings at /usr/local/dcgm/bindings/python3. No data will be captured.
xFormers 0.0.25
memory_efficient_attention.ckF:                    unavailable
memory_efficient_attention.ckB:                    unavailable
memory_efficient_attention.ck_decoderF:            unavailable
memory_efficient_attention.ck_splitKF:             unavailable
memory_efficient_attention.cutlassF:               available
memory_efficient_attention.cutlassB:               available
memory_efficient_attention.decoderF:               available
memory_efficient_attention.flshattF@v2.5.6:        available
memory_efficient_attention.flshattB@v2.5.6:        available
memory_efficient_attention.smallkF:                available
memory_efficient_attention.smallkB:                available
memory_efficient_attention.triton_splitKF:         unavailable
indexing.scaled_index_addF:                        unavailable
indexing.scaled_index_addB:                        unavailable
indexing.index_select:

In [4]:
token=""
from huggingface_hub import login, logout
login(token) # non-blocking login

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import LoraConfig
from trl import SFTTrainer
from transformers import TrainingArguments


import pandas as pd
df=pd.read_csv('/content/cleaned_data.csv')

# Device map
device_map = 'auto'  # for PP and running with `python test_sft.py`
# Load the model + tokenizer
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_cache=False,
    device_map=device_map
)
# PEFT config
lora_alpha = 16
lora_dropout = 0.1
lora_r = 32  # 64
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["k_proj", "q_proj", "v_proj", "up_proj", "down_proj", "gate_proj"],
    modules_to_save=["embed_tokens", "input_layernorm", "post_attention_layernorm", "norm"],
)
# Args
max_seq_length = 512
output_dir = "./results"
per_device_train_batch_size = 2  # reduced batch size to avoid OOM
gradient_accumulation_steps = 2 #2
optim = "adamw_torch"
save_steps = 20  # save every 20 steps
logging_steps = 1  # log every 1 step
learning_rate = 2e-4#2e-4
max_grad_norm = 0.3
max_steps = 5
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,  # gradient checkpointing
    #report_to="wandb",
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1384: UserWarning: Current model requires 1024.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [9]:
dataset=df

# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="context",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

# Memory statistics before training
gpu_statistics = torch.cuda.get_device_properties(0)
reserved_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 2)
max_memory = round(gpu_statistics.total_memory / 1024**3, 2)
print(f"Reserved Memory: {reserved_memory}GB")
print(f"Max Memory: {max_memory}GB")


# Train :)
trainer.train()

# Memory statistics after training
used_memory = round(torch.cuda.max_memory_allocated() / 1024**3, 2)
used_memory_lora = round(used_memory - reserved_memory, 2)
used_memory_persentage = round((used_memory / max_memory) * 100, 2)
used_memory_lora_persentage = round((used_memory_lora / max_memory) * 100, 2)
print(f"Used Memory: {used_memory}GB ({used_memory_persentage}%)")
print(f"Used Memory for training(fine-tuning) LoRA: {used_memory_lora}GB ({used_memory_lora_persentage}%)")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 14.75 GiB of which 869.06 MiB is free. Process 309616 has 13.90 GiB memory in use. Of the allocated memory 13.76 GiB is allocated by PyTorch, and 22.42 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

#LLM

In [1]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [2]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1
!pip3 install --upgrade bitsandbytes peft trl accelerate datasets transformers

  Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl (119.8 MB)
  Using cached peft-0.11.1-py3-none-any.whl (251 kB)
  Using cached trl-0.9.4-py3-none-any.whl (226 kB)
  Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
  Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
  Using cached transformers-4.42.3-py3-none-any.whl (9.3 MB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.17.0
    Uninstalling datasets-2.17.0:
      Successfully uninstalled datasets-2.17.0
  Attempting uninstall: 

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

model_id = "meta-llama/Meta-Llama-3-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, token=os.environ['HF_TOKEN'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
text = "Where can i get the best beer in dublin"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)
output = model.generate(**inputs, max_length=100, num_return_sequences=1)
#outputs = model.generate(**inputs, max_new_tokens=60)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Where can i get the best beer in dublin?
Where can i get the best beer in dublin?
Post by micko » 11:56am on 14 Aug 15
I know there are a lot of great places to drink in Dublin but i'm looking for somewhere that sells great beer, and i'm not talking about craft beer, i'm talking about great lagers, stouts, and ales. I'm from the UK so i'm not sure where


In [5]:
os.environ["WANDB_DISABLED"] = "true"
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)


In [6]:
import pandas as pd
df=pd.read_csv('/content/cleaned_data.csv')

In [7]:
df.head()

,place_name,review
0,14 Henrietta Street,museum good highly recommend know georgian hou...
1,1916 Rebellion Walking Tour,lorcan professional guide passionate irish his...
2,3fe Grand Canal Street,delicious coffee breakfast dublin lot indoor s...
3,Arthur Guinness Dwelling House,would think guinness beer time
4,As One,came saturday morning without booking got tabl...


In [8]:
# review = place name + review
df['review'] = df['place_name'] + ' ' + df['review']
df.head()


,place_name,review
0,14 Henrietta Street,14 Henrietta Street museum good highly recomme...
1,1916 Rebellion Walking Tour,1916 Rebellion Walking Tour lorcan professiona...
2,3fe Grand Canal Street,3fe Grand Canal Street delicious coffee breakf...
3,Arthur Guinness Dwelling House,Arthur Guinness Dwelling House would think gui...
4,As One,As One came saturday morning without booking g...


In [39]:
#!pip install transformers trl torch pandas scikit-learn
#!pip install --upgrade tral[pyarrow]
!pip install trl

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
